In [1]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

ModuleNotFoundError: No module named 'fastai'

In [2]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:77]

'Once again Mr. Costner has dragged out a movie for far longer than necessary.'

Word Tokenization

In [7]:
#| label: word-tokenizer
#| code-fold: true
spacy = WordTokenizer()
toks = first(spacy([txt[:77]]))
print(coll_repr(toks, 30))

(#15) ['Once','again','Mr.','Costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.']


In [9]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [10]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','xxmaj','aside','from','the','terrific','sea','rescue','sequences',',','of','which'...]


In [11]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [12]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Subword Tokenization

In [13]:
txts = L(o.open().read() for o in files[:2000])

In [14]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [17]:
subword(1000)

'▁O n ce ▁again ▁M r . ▁Co st n er ▁has ▁ d ra g g ed ▁out ▁a ▁movie ▁for ▁far ▁long er ▁than ▁ ne ce s s ar y . ▁A side ▁from ▁the ▁ ter'

In [18]:
subword(200)

'▁ O n ce ▁a g a in ▁ M r . ▁ C o s t n er ▁ h a s ▁ d ra g g ed ▁ o u t ▁a ▁movie ▁for ▁f ar ▁ l'

In [19]:
subword(10000)

'▁On ce ▁again ▁Mr . ▁Costner ▁has ▁dragged ▁out ▁a ▁movie ▁for ▁far ▁longer ▁than ▁necessary . ▁A side ▁from ▁the ▁terrific ▁sea ▁rescue ▁sequences , ▁of ▁which ▁there ▁are ▁very ▁few ▁I ▁just ▁did ▁not ▁care ▁about ▁any ▁of'

Numericalization with fastai

In [20]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','xxmaj','aside','from','the','terrific','sea','rescue','sequences',',','of','which'...]


In [21]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has'...]

In [22]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1968) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','it','i','in'...]"

In [23]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,  349,  183,    8, 1177,   10,    8, 1178,   60, 1455,
              62,   12,   25,   28,  189,  957,   93,  958,   10])

In [24]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj once again xxmaj mr . xxmaj costner has dragged out a movie for far longer than necessary .'

Putting Our Texts into Batches for a Language Model

In [29]:
from weasyprint import HTML

#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))


-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'gobject-2.0-0': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'gobject-2.0-0'